# Gradient Descent Optimization Algorithms
In this notebook you can find a collection of GD based optimization algorithm used for deep learning. The code is always accompanied by a explanatory youtube video which are linked here:
- [Stochastic Gradient Descent](https://youtu.be/IH9kqpMORLM)
- [Stochastic Gradient Descent + Momentum](https://youtu.be/7EuiXb6hFAM)
- [Adagrad](https://youtu.be/EGt-UOIIdDk)
- [RMSprop](https://youtu.be/nLCuzsQaAKE)
- [AdaDelta](https://youtu.be/6gvh0IySNMs)
- [Adam](https://youtu.be/6nqV58NA_Ew)
- [Nesterov](https://youtu.be/6FrBXv9OcqE)
- [Adamax](https://youtu.be/Uef4BofnVn0)
- [Nadam](https://youtu.be/8nfd7gEDKCc)
- QHM and QHAdam

## Tests
In order to demonstrate the algorithms capabilities to optimize a function we used these simple test setup:
- learning various linear function of the form `f(x) = w0 + w1*x` with the squared error. This is a simple sanity check as the gradient are simple to calculate and the test data is also easy to generate.

In [1]:
import numpy as np
import copy
from numpy.random import permutation

class Line():
    """
        Linear Model with two weights w0 (intercept) and w1 (slope)
    """
    def __init__(self):
        self.weights = [np.random.uniform(0,1,1) for _ in range(2)]
        self.derivative_funcs = [self.dx_w0, self.dx_w1]
        
    def evaluate(self,x):
        """
            evaluate: will evaluate the line yhate given x
            x: a point in the plane

            return the result of the function evalutation
        """
        return self.weights[0] + self.weights[1]*x

    def derivate(self, x, y):
        """
            derivate: will calculate all partial derivatives and return them
            input:
            x: a point in the plane
            y: the response of the point x
            
            output:
            partial_derivatives: an array of partial derivatives
        """
        partial_derivatives = []
        
        yhat = self.evaluate(x)
        partial_derivatives.append(self.dx_w0(x, y, yhat))
        partial_derivatives.append(self.dx_w1(x, y, yhat))
        
        return partial_derivatives
    
    def dx_w0(self, x, y, yhat):
        """
            dx_w0: partial derivative of the weight w0
            x: a point in the plane
            y: the response of the point x
            yhat: the current approximation of y given x and the weights

            return the gradient at that point for this x and y for w0
        """
        return 2*(yhat - y)
    
    def dx_w1(self, x, y, yhat):
        """
            dx_w1: partial derivative of the weight w1 for a linear function
            x: a point in the plane
            y: the response of the point x
            yhat: the current approximation of y given x and the weights

            return the gradient at that point for this x and y for w1
        """  
        return 2*x*(yhat - y)

    def __str__(self):
        return f"y = {self.weights[0]} + {self.weights[1]}*x"
        
    
#################### Helper functions ######################
def stochastic_sample(xs, ys):
    """
        stochastic_sample: sample with replacement one x and one y
        xs: all point on the plane
        ys: all response on the plane
        
        return the randomly selected x and y point
    """
    perm = permutation(len(xs))
    x = xs[perm[0]]
    y = ys[perm[0]]

    return x, y
    
    
def gradient(dx, evaluate, xs, ys):
    """
        gradient: estimate mean gradient over all point for w1
        evaluate: the evaulation function from the model
        dx: partial derivative function used to evaluate the gradient
        xs: all point on the plane
        ys: all response on the plane
        
        return the mean gradient all x and y for w1
    """         
    N = len(ys)
    
    total = 0
    for x,y in zip(xs,ys):
        yhat = evaluate(x)
        total = total + dx(x, y, yhat)
    
    gradient = total/N
    return gradient

################## Optimization Functions #####################

def gd(model, xs, ys, learning_rate = 0.01, max_num_iteration = 1000):
    """
        gd: will estimate the parameters w1 and w2 (here it uses least square cost function)
        model: the model we are trying to optimize using gradient descent
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
    """    

    for i in range(max_num_iteration):
        # Updating the model parameters
        model.weights = [weight - learning_rate*gradient(derivative_func, model.evaluate, xs, ys) for weight, derivative_func in zip(model.weights, model.derivative_funcs)]
        
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
def sgd(model, xs, ys, learning_rate = 0.01, max_num_iteration = 1000):
    """
        sgd: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
    """       
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        # Updating the model parameters
        model.weights = [weight - learning_rate*derivative for weight, derivative in zip(model.weights, model.derivate(x,y))]        
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
def sgd_momentum(model, xs, ys, learning_rate = 0.01, decay_factor = 0.9, max_num_iteration = 1000):
    """
        sgd_momentum: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        decay_factor: determines the relative contribution of the current gradient and earlier gradients to the weight change
        max_num_iteration: the number of iteration before we stop updating
    """
    
    # Create the gradient that we keep track as an array of 0 of the same size as the number of weights
    gradients = [0 for _ in range(len(model.weights))]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)

        # Calculate the new gradients
        gradients = [decay_factor*g + learning_rate*derivative for g, derivative in zip(gradients, model.derivate(x,y))]
        
        # Updating the model parameters
        model.weights = [weight - g for weight, g in zip(model.weights, gradients)]
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
            
def adagrad(model, xs, ys, learning_rate = 0.1, max_num_iteration = 1000, eps=0.0000001):
    """
        adagrad: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        max_num_iteration: the number of iteration before we stop updating
        eps: is a numerical safety to avoid division by 0
    """         
    
    # Here only the diagonal matter
    num_param = len(model.weights)
    G = [[0 for _ in range(num_param)] for _ in range(num_param)]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        # Update G and the model weights iteratively (Note: speed up could be gained from vectorized implementation)
        for idx, gradient in enumerate(model.derivate(x, y)):
            G[idx][idx] = G[idx][idx] + gradient**2
            model.weights[idx] = model.weights[idx] - (learning_rate / np.sqrt(G[idx][idx] + eps)) * gradient
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
def rmsprop(model, xs, ys, learning_rate = 0.01, decay_factor = 0.9, max_num_iteration = 10000, eps=0.0000001):
    """
        rmsprop: will estimate the parameters w0 and w1 
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        decay_factor: the parameter used in the running averaging
        max_num_iteration: the number of iteration before we stop updating
        eps: is a numerical safety to avoid division by 0
    """         
    
    # Running average
    E = [0 for _ in range(len(model.weights))]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        # Update E and the model weights iteratively (Note: speed up could be gained from vectorized implementation)
        for idx, gradient in enumerate(model.derivate(x, y)):    
            E[idx] = decay_factor*E[idx] + (1 - decay_factor)*(gradient**2)
            model.weights[idx] = model.weights[idx] - (learning_rate/np.sqrt(E[idx] + eps))*gradient

    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)

def adadelta(model, xs, ys, decay_factor = 0.9, max_num_iteration = 10000, eps=0.0000001):
    """
        Adadelta: will estimate the parameters w0 and w1
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        decay_factor: the parameter used in the running averaging
        max_num_iteration: the number of iteration before we stop updating
        eps: is a numerical safety to avoid division by 0
    """         
    
    # Init Running Averages
    num_param = len(model.weights)
    E_g = [0 for _ in range(num_param)]
    E_p = [0 for _ in range(num_param)]
    delta_p = [0 for _ in range(num_param)]
    
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)
        
        for idx, gradient in enumerate(model.derivate(x, y)):
            # Get the running average for the gradient
            E_g[idx] = decay_factor*E_g[idx] + (1 - decay_factor)*(gradient**2)
            
            # Get the running average for the parameters
            E_p[idx] = decay_factor*E_p[idx] + (1 - decay_factor)*(delta_p[idx]**2)
            
            # Calculate the gradient difference
            delta_p[idx] = - np.sqrt(E_p[idx] + eps) / np.sqrt(E_g[idx] + eps) * gradient
            
            # update the model weight
            model.weights[idx] = model.weights[idx] + delta_p[idx]
            
        
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            

def adam(model, xs, ys, learning_rate = 0.1, b1 = 0.9, b2 = 0.999, epsilon = 0.00000001, max_iteration = 1000):
    """
        Adam: This is the adam optimizer that build upon adadelta and RMSProp
        model: The model we want to optimize the parameter on
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the amount of learning we want to happen at each time step (default is 0.1 and will be updated by the optimizer)
        b1: this is the first decaying average with proposed default value of 0.9 (deep learning purposes)
        b2: this is the second decaying average with proposed default value of 0.999 (deep learning purposes)
        epsilon: a variable for numerical stability during the division
        max_iteration: the number of sgd round we want to do before stopping the optimization
    """
    
    
    # Variable Initialization
    num_param = len(model.weights)
    m = [0 for _ in range(num_param)] # two m for each parameter
    v = [0 for _ in range(num_param)] # two v for each parameter
    g = [0 for _ in range(num_param)] # two gradient
    
    for t in range(1,max_iteration):
        
        # Calculate the gradients 
        x, y = stochastic_sample(xs, ys)
        
        # Get the partial derivatives
        g = model.derivate(x, y)

        # Update the m and v parameter
        m = [b1*m_i + (1 - b1)*g_i for m_i, g_i in zip(m, g)]
        v = [b2*v_i + (1 - b2)*(g_i**2) for v_i, g_i in zip(v, g)]

        # Bias correction for m and v
        m_cor = [m_i / (1 - (b1**t)) for m_i in m]
        v_cor = [v_i / (1 - (b2**t)) for v_i in v]

        # Update the parameter
        model.weights = [weight - (learning_rate / (np.sqrt(v_cor_i) + epsilon))*m_cor_i for weight, v_cor_i, m_cor_i in zip(model.weights, v_cor, m_cor)]
        
        if t % 100 == 0:
            print(f"Iteration {t}")
            print(model)
    
def nesterov(model, xs, ys, learning_rate = 0.01, decay_factor = 0.9, max_num_iteration = 1000):
    """
        Nesterov: This is the nesterov accelerated gradient optimizer that build upon momentum
        model: the model we want to optimize the parameter on (this is a line right now)
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the learning rate for the step that weights update will take
        decay_factor: determines the relative contribution of the current gradient and earlier gradients to the weight change
        max_num_iteration: the number of iteration before we stop updating
    """
    
    # These are needed to keep track of the previous gradient
    g = [0 for _ in range(len(model.weights))] 
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)

        # Calculate the gradient for w0 by predicting where the ball will be (approximatively)
        for idx, gradient in enumerate(model.derivate(x,y)):
            
            # Here we need to do a bit of gymnastic because of how the code is setup
            # We need to save the parameters state, modify it, do the simulation and then reset the parameter state
            # The update happen in the next section
            prev_weight = model.weights[idx]
            model.weights[idx] = decay_factor*gradient
            g[idx] = decay_factor*g[idx] + learning_rate*gradient
            model.weights[idx] = prev_weight
            
            # Update the model parameter
            model.weights[idx] = model.weights[idx] - g[idx]
            
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)
            
            
def adamax(model, xs, ys, learning_rate = 0.1, b1 = 0.9, b2 = 0.999, max_iteration = 1000):
    """
        Adamax: This is the adamax optimizer that build upong adam with L_inf norm
        model: The model we want to optimize the parameter on
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the amount of learning we want to happen at each time step (default is 0.1 and will be updated by the optimizer)
        b1: this is the first decaying average with proposed default value of 0.9 (deep learning purposes)
        b2: this is the second decaying average with proposed default value of 0.999 (deep learning purposes)
        max_iteration: the number of sgd round we want to do before stopping the optimization
    """
    
    
    # Variable Initialization
    num_param = len(model.weights)
    m = [0 for _ in range(num_param)] # two m for each parameter
    v = [0 for _ in range(num_param)] # two v for each parameter
    g = [0 for _ in range(num_param)] # two gradient
    
    for t in range(1,max_iteration):
        
        # Calculate the gradients 
        x, y = stochastic_sample(xs, ys)
        
        # Get the partial derivatives
        g = model.derivate(x, y)

        # Update the m and v parameter
        m = [b1*m_i + (1 - b1)*g_i for m_i, g_i in zip(m, g)]
        v = [np.maximum(b2*v_i, np.absolute(g_i)) for v_i, g_i in zip(v, g)]

        # Bias correction for m only
        m_cor = [m_i / (1 - (b1**t)) for m_i in m]

        # Update the parameter
        model.weights = [weight - (learning_rate / np.sqrt(v_i))*m_cor_i for weight, v_i, m_cor_i in zip(model.weights, v, m_cor)]
        
        if t % 100 == 0:
            print(f"Iteration {t}")
            print(model)
            
def nadam(model, xs, ys, learning_rate = 0.1, b1 = 0.9, b2 = 0.999, epsilon = 0.00000001, max_iteration = 1000):
    """
        Nadam: This is the adam optimizer that build upon adadelta and RMSProp with Nesterov Accelerated Gradient sprinkled on top
        model: The model we want to optimize the parameter on
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the amount of learning we want to happen at each time step (default is 0.1 and will be updated by the optimizer)
        b1: this is the first decaying average with proposed default value of 0.9 (deep learning purposes)
        b2: this is the second decaying average with proposed default value of 0.999 (deep learning purposes)
        epsilon: a variable for numerical stability during the division
        max_iteration: the number of sgd round we want to do before stopping the optimization
    """
    
    
    # Variable Initialization
    num_param = len(model.weights)
    m = [0 for _ in range(num_param)] # two m for each parameter
    v = [0 for _ in range(num_param)] # two v for each parameter
    g = [0 for _ in range(num_param)] # two gradient
    
    for t in range(1,max_iteration):
        
        # Calculate the gradients 
        x, y = stochastic_sample(xs, ys)
        
        # Get the partial derivatives
        g = model.derivate(x, y)

        # Update the m and v parameter
        m = [b1*m_i + (1 - b1)*g_i for m_i, g_i in zip(m, g)]
        v = [b2*v_i + (1 - b2)*(g_i**2) for v_i, g_i in zip(v, g)]

        # Bias correction for m and v
        m_cor = [m_i / (1 - (b1**t)) for m_i in m]
        v_cor = [v_i / (1 - (b2**t)) for v_i in v]

        # nesterov look-ahead gradient
        nag_g = [b1*m_cor_i + ((1-b1) * g_i)/(1 - b1**t) for m_cor_i, g_i in zip(m_cor, g)]
        
        # Update the parameter
        model.weights = [weight - (learning_rate / (np.sqrt(v_cor_i) + epsilon))*nag_g_i for weight, v_cor_i, nag_g_i in zip(model.weights, v_cor, nag_g)]
        
        if t % 100 == 0:
            print(f"Iteration {t}")
            print(model)

def qhm(model, xs, ys, learning_rate = 0.01, decay_factor = 0.999, v = 0.7, max_num_iteration = 1000):
    """
        QHM: will estimate the parameters w0 and w1 using a weighted average between SGD and Momentum.
        (here it uses least square cost function)
        model: the model we are trying to optimize using sgd
        xs: all point on the plane
        ys: all response on the plane
        learning_rate: the learning rate for the step that weights update will take
        v: immediate discount factor ν
        decay_factor: determines the relative contribution of the current gradient and earlier gradients to the weight change
        max_num_iteration: the number of iteration before we stop updating
    """
    
    # Create the gradient that we keep track as an array of 0 of the same size as the number of weights
    moment_buffer = [0 for _ in range(len(model.weights))]
    
    for i in range(max_num_iteration):
        
        # Select a random x and y
        x, y = stochastic_sample(xs, ys)

        # Calculate the new gradients
        gradients = model.derivate(x,y)
        moment_buffer = [decay_factor*g + (1-decay_factor)*grad for g, grad in zip(moment_buffer, gradients)]
        
        # Updating the model parameters
        model.weights = [weight - learning_rate*((1-v)*grad + v*g) for weight, g, grad in zip(model.weights, moment_buffer, gradients)]
    
        if i % 100 == 0:
            print(f"Iteration {i}")
            print(model)

def qhadam(model, xs, ys, learning_rate = 0.1, b1 = 0.95, b2 = 0.98, v1 = 0.8, v2 = 0.7, epsilon = 0.00000001, max_iteration = 1000):
    """
        QhAdam: This is the quasi hyperbolic modification of the adam optimizer (can recover lots of other form of Adam)
        model: The model we want to optimize the parameter on
        xs: the feature of my dataset
        ys: the continous value (target)
        learning_rate: the amount of learning we want to happen at each time step (default is 0.1 and will be updated by the optimizer)
        b1: this is the first decaying average with proposed default value of 0.9 (deep learning purposes)
        b2: this is the second decaying average with proposed default value of 0.999 (deep learning purposes)
        v1: immediate discount factor for the first moment vector
        v2: immediate discount factor for the second moment vector
        epsilon: a variable for numerical stability during the division
        max_iteration: the number of sgd round we want to do before stopping the optimization
    """
    
    
    # Variable Initialization
    num_param = len(model.weights)
    m = [0 for _ in range(num_param)] # two m for each parameter
    v = [0 for _ in range(num_param)] # two v for each parameter
    g = [0 for _ in range(num_param)] # two gradient
    
    for t in range(1,max_iteration):
        
        # Calculate the gradients 
        x, y = stochastic_sample(xs, ys)
        
        # Get the partial derivatives
        g = model.derivate(x, y)

        # Update the m and v parameter
        m = [b1*m_i + (1 - b1)*g_i for m_i, g_i in zip(m, g)]
        v = [b2*v_i + (1 - b2)*(g_i**2) for v_i, g_i in zip(v, g)]

        # Bias correction for m and v
        m_cor = [m_i / (1 - (b1**t)) for m_i in m]
        v_cor = [v_i / (1 - (b2**t)) for v_i in v]

        # Update the parameter
        model.weights = [weight - learning_rate*((1-v1)*g_i + v1*m_cor_i / (np.sqrt((1-v2)*(g_i**2) + v2*v_cor_i)) + epsilon) for weight, v_cor_i, m_cor_i, g_i in zip(model.weights, v_cor, m_cor, g)]
        
        if t % 100 == 0:
            print(f"Iteration {t}")
            print(model)

In [2]:
# Here we have a simple line with intercept = 0 and slope = 1
xs = [1,2,3,4,5,6,7]
ys = [1,2,3,4,5,6,7]

print("Target: intercept = 0 and slope = 1")

'''
# Gradient Descent
model = Line()
print("Gradient Descent: ")
gd(model, xs, ys)
print(model)

# Stochastic Gradient Descent
model = Line()
print("Stochastic Gradient Descent: ")
sgd(model, xs, ys)
print(model)

# Stochastic Gradient Descent with Momentum
model = Line()
print("SGD + Momentum: ")
sgd_momentum(model, xs, ys)
print(model)


# Adagrad
model = Line()
print("Adagrad")
adagrad(model, xs, ys)
print(model)

# RMSprop
model = Line()
print("RMSprop")
rmsprop(model, xs, ys)
print(model)

# Adadelta
model = Line()
print("Adadelta")
adadelta(model, xs, ys)
print(model)

# Adam
model = Line()
print("Adam")
adam(model, xs, ys)
print(model)

# Nesterov Accelerated Gradient
model = Line()
print("Nesterov Accelerated Gradient")
nesterov(model, xs, ys)
print(model)

# Adamax
model = Line()
print("Adamax")
adamax(model, xs, ys)
print(model)

# Nadam
model = Line()
print("Nadam")
nadam(model, xs, ys)
print(model)


# QHM
model = Line()
print("QHM")
qhm(model, xs, ys)
print(model)
'''

# QHAdam
model = Line()
print("QHAdam")
qhadam(model, xs, ys)
print(model)

Target: intercept = 0 and slope = 1
QHAdam
Iteration 100
y = [-0.01745708] + [1.00340449]*x
Iteration 200
y = [0.00197568] + [0.99960321]*x
Iteration 300
y = [-0.00020042] + [1.000061]*x
Iteration 400
y = [-1.51876833e-06] + [0.99999851]*x
Iteration 500
y = [0.02677531] + [0.98893378]*x
Iteration 600
y = [0.00152994] + [1.00348552]*x
Iteration 700
y = [0.00819687] + [1.00075611]*x
Iteration 800
y = [-0.02847251] + [0.99875309]*x
Iteration 900
y = [-0.00252884] + [0.99969617]*x
y = [-0.02410363] + [1.00200414]*x


In [3]:

# Here we have a simple line with intercept = 0 and slope = 2
xs = [1,2,3,4,5,6,7]
ys = [2,4,6,8,10,12,14]

print("Target: intercept = 0 and slope = 2")

'''
# Gradient Descent
model = Line()
print("Gradient Descent: ")
gd(model, xs, ys)
print(model)

# Stochastic Gradient Descent
model = Line()
print("Stochastic Gradient Descent: ")
sgd(model, xs, ys)
print(model)

# Stochastic Gradient Descent with Momentum
model = Line()
print("SGD + Momentum: ")
sgd_momentum(model, xs, ys)
print(model)

# Adagrad
model = Line()
print("Adagrad")
adagrad(model, xs, ys)
print(model)

# RMSprop
model = Line()
print("RMSprop")
rmsprop(model, xs, ys)
print(model)

# Adadelta
model = Line()
print("Adadelta")
adadelta(model, xs, ys)
print(model)

# Adam
model = Line()
print("Adam")
adam(model, xs, ys)
print(model)

# Nesterov Accelerated Gradient
model = Line()
print("Nesterov Accelerated Gradient")
nesterov(model, xs, ys)
print(model)

# Adamax
model = Line()
print("Adamax")
adamax(model, xs, ys)
print(model)

# QHM
model = Line()
print("QHM")
qhm(model, xs, ys)
print(model)
'''

# QHAdam
model = Line()
print("QHAdam")
qhadam(model, xs, ys)
print(model)

Target: intercept = 0 and slope = 2
QHAdam
Iteration 100
y = [-0.03911129] + [2.00762027]*x
Iteration 200
y = [0.00706794] + [1.9978334]*x
Iteration 300
y = [-0.00040877] + [2.00018629]*x
Iteration 400
y = [6.53100208e-06] + [2.00002451]*x
Iteration 500
y = [-0.03343831] + [2.00452959]*x
Iteration 600
y = [0.00540891] + [2.00012642]*x
Iteration 700
y = [-0.02074989] + [2.00274253]*x
Iteration 800
y = [0.06844824] + [1.998662]*x
Iteration 900
y = [-0.01387991] + [2.00180325]*x
y = [-0.00901815] + [1.99258471]*x


In [4]:
# Here we have a simple line with intercept = 1 and slope = 2
xs = [1,2,3,4,5,6,7]
ys = [3,5,7,9,11,13,15]

print("Target: intercept = 1 and slope = 2")

'''
# Gradient Descent
model = Line()
print("Gradient Descent: ")
gd(model, xs, ys)
print(model)

# Stochastic Gradient Descent
model = Line()
print("Stochastic Gradient Descent: ")
sgd(model, xs, ys)
print(model)

# Stochastic Gradient Descent with Momentum
model = Line()
print("SGD + Momentum: ")
sgd_momentum(model, xs, ys)
print(model)

# Adagrad
model = Line()
print("Adagrad")
adagrad(model, xs, ys)
print(model)

# RMSprop
model = Line()
print("RMSprop")
rmsprop(model, xs, ys)
print(model)

# Adadelta
model = Line()
print("Adadelta")
adadelta(model, xs, ys)
print(model)

# Adam
model = Line()
print("Adam")
adam(model, xs, ys)
print(model)

# Nesterov Accelerated Gradient
model = Line()
print("Nesterov Accelerated Gradient")
nesterov(model, xs, ys)
print(model)

# Adamax
model = Line()
print("Adamax")
adamax(model, xs, ys)
print(model)

# QHM
model = Line()
print("QHM")
qhm(model, xs, ys)
print(model)
'''

# QHAdam
model = Line()
print("QHAdam")
qhadam(model, xs, ys)
print(model)

Target: intercept = 1 and slope = 2
QHAdam
Iteration 100
y = [0.9993348] + [2.00089617]*x
Iteration 200
y = [1.00160173] + [1.99937833]*x
Iteration 300
y = [0.99994153] + [2.00001534]*x
Iteration 400
y = [0.99998916] + [1.99999895]*x
Iteration 500
y = [1.00181985] + [1.99885023]*x
Iteration 600
y = [1.00896344] + [1.99764551]*x
Iteration 700
y = [1.00896808] + [2.00543252]*x
Iteration 800
y = [1.04140175] + [1.98514876]*x
Iteration 900
y = [1.02124556] + [2.00204173]*x
y = [1.00288658] + [2.00121553]*x
